# Recolección de imágenes satelitales

## Análisis de funcionalidad para posible uso

Importación de librerías necesarias

In [2]:
import pandas as pd
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import os
import re

Cargar csv 

In [3]:
# Cargamos el archivo con las fechas y horas
ruta_eventos = "../data/raw/evento_granizo_limpio_hora.csv"
df_eventos = pd.read_csv(ruta_eventos, parse_dates=['fecha'])

print("✅ Archivo de eventos con horas cargado exitosamente.")
display(df_eventos.head())

✅ Archivo de eventos con horas cargado exitosamente.


,fecha,ubicacion,fuente,estacion_asignada,hora_aprox
0,2000-01-01,Aeropuerto El Plumerillo (Norte de Mendoza),researchgate.net,"MENDOZA AERO, AR",22:45 - 23:20 UTC
1,2002-03-08,Puente de Hierro (Guaymallén),argentina.gob.ar,"MENDOZA AERO, AR",NaN
2,2005-02-14,Luján de Cuyo,eluniverso.com,"MENDOZA AERO, AR",Tarde
3,2006-02-12,"Luján de Cuyo (Perdriel, Vistalba)",Diario Los Andes,"MENDOZA AERO, AR",NaN
4,2008-01-31,Ciudad de Mendoza,mdzol.com,"MENDOZA AERO, AR",20:20 - 20:37 hs


Verificamos los horarios únicos del dataset

In [4]:
# Obtenemos y mostramos todos los valores únicos de la columna 'hora_aprox'
valores_unicos_hora = df_eventos['hora_aprox'].unique()

print("--- Horarios Únicos Encontrados en tu CSV ---")
print(valores_unicos_hora)

--- Horarios Únicos Encontrados en tu CSV ---
['22:45 - 23:20 UTC' nan 'Tarde' '20:20 - 20:37 hs'
 'Cerca de las 14:00 hs' '18:20 hs' 'Mediodía' 'Tarde (aprox. 18:00 hs)'
 'Tarde (después de las 16:00 hs)' '08:30 hs' 'Noche (21:00 - 01:00 hs)'
 'Noche' 'Tarde (desde las 13:00 hs)' 'Madrugada'
 'Tarde-Noche (15:00 - 19:00 hs)' 'Tarde-Noche' '17:15 hs'
 'Tarde-Noche (aprox. 20:00 hs)' 'Tarde (16:00 hs)' '16:00 - 21:30 hs'
 'Tarde (desde las 16:00 hs)' 'Tarde (desde las 16:45 hs)'
 'Noche (aprox. 23:00 hs)' 'Madrugada (aprox. 01:30 hs)']


In [ ]:
Script para obtener imagenes

1) La función traductora de horas

In [5]:
def parse_hora_a_utc(texto_hora):
    if pd.isna(texto_hora):
        return None # Si no hay dato, devolvemos Nada
    
    texto_hora = str(texto_hora).lower()
    
    # Caso 1: La hora ya está en UTC
    if 'utc' in texto_hora:
        numeros = re.findall(r'\d+', texto_hora)
        if numeros:
            return int(numeros[0]) # Devuelve la primera hora que encuentre
            
    # Caso 2: Horas de texto
    if 'tarde' in texto_hora:
        return 20 # 17:00 hs local -> 20:00 UTC
    if 'noche' in texto_hora:
        return 1 # 22:00 hs local -> 01:00 UTC del día siguiente (simplificado)
    if 'madrugada' in texto_hora:
        return 5 # 02:00 hs local -> 05:00 UTC
    if 'mediodía' in texto_hora or 'mediodia' in texto_hora:
        return 15 # 12:00 hs local -> 15:00 UTC
        
    # Caso 3: Extraer el primer número de la cadena
    numeros = re.findall(r'\d+', texto_hora)
    if numeros:
        hora_local = int(numeros[0])
        # Convertimos a UTC y manejamos el cambio de día
        hora_utc = (hora_local + 3) % 24
        return hora_utc
        
    return None # Si no se pudo interpretar, devolvemos Nada


2) Configuración para la descarga

In [6]:
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))
bucket_name = 'noaa-goes16'
ruta_guardado = "../data/raw/imagenes_satelitales/"
os.makedirs(ruta_guardado, exist_ok=True)
print("Iniciando la descarga de imágenes satelitales...")

Iniciando la descarga de imágenes satelitales...


3) Bucle principal 

In [11]:
for index, evento in df_eventos.iterrows():
    # Usamos nuestra nueva función para obtener la hora UTC
    hora_utc = parse_hora_a_utc(evento['hora_aprox'])
    
    # Solo procedemos si obtuvimos una hora válida
    if hora_utc is not None:
        try:
            fecha = evento['fecha']
            año = fecha.year
            dia_del_año = fecha.dayofyear
            
            prefix = f"ABI-L2-CMIPF/{año}/{dia_del_año:03d}/{hora_utc:02d}/"
            response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
            
            if 'Contents' in response:
                key_archivo = response['Contents'][0]['Key']
                nombre_archivo_local = f"{ruta_guardado}{fecha.strftime('%Y-%m-%d')}_{hora_utc:02d}00_UTC.nc"
                
                if not os.path.exists(nombre_archivo_local):
                    print(f"-> Descargando imagen para {fecha.strftime('%Y-%m-%d')} a las {hora_utc:02d}h UTC...")
                    s3.download_file(bucket_name, key_archivo, nombre_archivo_local)
                    print(f"✅ Guardado como: {nombre_archivo_local}")
                else:
                    print(f"-> Archivo ya existe: {nombre_archivo_local}. Saltando.")
            else:
                print(f"-> No se encontraron imágenes para {fecha.strftime('%Y-%m-%d')} a las {hora_utc:02d}:00 UTC.")
        except Exception as e:
            print(f"-> ERROR procesando el evento del {evento['fecha'].strftime('%Y-%m-%d')}: {e}")

print("\n--- Proceso de descarga finalizado ---")

-> No se encontraron imágenes para 2000-01-01 a las 22:00 UTC.
-> No se encontraron imágenes para 2005-02-14 a las 20:00 UTC.
-> No se encontraron imágenes para 2008-01-31 a las 23:00 UTC.
-> No se encontraron imágenes para 2008-11-26 a las 17:00 UTC.
-> No se encontraron imágenes para 2008-12-18 a las 21:00 UTC.
-> No se encontraron imágenes para 2010-09-28 a las 15:00 UTC.
-> No se encontraron imágenes para 2011-02-23 a las 20:00 UTC.
-> No se encontraron imágenes para 2011-11-20 a las 20:00 UTC.
-> No se encontraron imágenes para 2012-10-29 a las 11:00 UTC.
-> No se encontraron imágenes para 2013-02-09 a las 01:00 UTC.
-> No se encontraron imágenes para 2014-10-26 a las 01:00 UTC.
-> No se encontraron imágenes para 2014-12-09 a las 20:00 UTC.
-> No se encontraron imágenes para 2014-12-16 a las 05:00 UTC.
-> No se encontraron imágenes para 2015-09-25 a las 20:00 UTC.
-> No se encontraron imágenes para 2016-03-01 a las 01:00 UTC.
-> Descargando imagen para 2017-04-05 a las 20h UTC...


### Solución a errores previos en la descarga

In [10]:
import pandas as pd
import boto3
from botocore import UNSIGNED  # <-- CORRECCIÓN DEL ERROR DE TIPEO
from botocore.config import Config
import os
import re
import time

# --- 1. Cargar tu archivo de eventos con las horas ---
try:
    ruta_eventos = "../data/raw/evento_granizo_limpio_hora.csv"
    df_eventos = pd.read_csv(ruta_eventos, parse_dates=['fecha'])
    print("✅ Archivo de eventos con horas cargado exitosamente.")
except FileNotFoundError:
    print(f"❌ Error: No se encontró el archivo en la ruta '{ruta_eventos}'.")
    df_eventos = None # Detenemos si no se encuentra el archivo

if df_eventos is not None:
    # --- 2. Instalar las librerías necesarias ---
    !pip install boto3
    
    # --- 3. La Función "Traductora" de Horas ---
    def parse_hora_a_utc(texto_hora):
        if pd.isna(texto_hora):
            return None
        texto_hora = str(texto_hora).lower()
        if 'utc' in texto_hora:
            numeros = re.findall(r'\d+', texto_hora)
            return int(numeros[0]) if numeros else None
        if 'tarde' in texto_hora: return 20 # 17:00 ART -> 20:00 UTC
        if 'noche' in texto_hora: return 1  # 22:00 ART -> 01:00 UTC (+1 día)
        if 'madrugada' in texto_hora: return 5 # 02:00 ART -> 05:00 UTC
        if 'mediodía' in texto_hora or 'mediodia' in texto_hora: return 15 # 12:00 ART -> 15:00 UTC
        numeros = re.findall(r'\d+', texto_hora)
        if numeros:
            hora_local = int(numeros[0])
            return (hora_local + 3) % 24
        return None

    # --- 4. Configuración para la descarga ---
    s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))
    bucket_name = 'noaa-goes16'
    ruta_guardado = "../data/raw/imagenes_satelitales/"
    os.makedirs(ruta_guardado, exist_ok=True)
    print("\nIniciando la descarga de imágenes satelitales...")

    # --- 5. Bucle principal de descarga ---
    for index, evento in df_eventos.iterrows():
        hora_utc = parse_hora_a_utc(evento['hora_aprox'])
        
        if hora_utc is not None:
            try:
                fecha = evento['fecha']
                año = fecha.year
                dia_del_año = fecha.dayofyear
                
                # Buscamos archivos de la Banda 13 (CMIPF)
                prefix = f"ABI-L2-CMIPF/{año}/{dia_del_año:03d}/{hora_utc:02d}/"
                response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
                
                archivo_encontrado = None
                if 'Contents' in response:
                    for file in response['Contents']:
                        if 'C13' in file['Key']: # Buscamos el identificador de la Banda 13
                            archivo_encontrado = file['Key']
                            break
                
                if archivo_encontrado:
                    nombre_archivo_local = f"{ruta_guardado}{fecha.strftime('%Y-%m-%d')}_{hora_utc:02d}00_UTC_C13.nc"
                    if not os.path.exists(nombre_archivo_local):
                        print(f"-> Descargando: {archivo_encontrado}")
                        s3.download_file(bucket_name, archivo_encontrado, nombre_archivo_local)
                        print(f"✅ Guardado como: {nombre_archivo_local}")
                    else:
                        print(f"-> Archivo ya existe: {nombre_archivo_local}. Saltando.")
                else:
                    print(f"-> No se encontró la Banda 13 para {fecha.strftime('%Y-%m-%d')} a las {hora_utc:02d}:00 UTC.")

            except Exception as e:
                print(f"-> ERROR procesando el evento del {evento['fecha'].strftime('%Y-%m-%d')}: {e}")
            
            time.sleep(1) # Pequeña pausa de cortesía

    print("\n--- Proceso de descarga finalizado ---")

✅ Archivo de eventos con horas cargado exitosamente.

Iniciando la descarga de imágenes satelitales...



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


-> No se encontró la Banda 13 para 2000-01-01 a las 22:00 UTC.
-> No se encontró la Banda 13 para 2005-02-14 a las 20:00 UTC.
-> No se encontró la Banda 13 para 2008-01-31 a las 23:00 UTC.
-> No se encontró la Banda 13 para 2008-11-26 a las 17:00 UTC.
-> No se encontró la Banda 13 para 2008-12-18 a las 21:00 UTC.
-> No se encontró la Banda 13 para 2010-09-28 a las 15:00 UTC.
-> No se encontró la Banda 13 para 2011-02-23 a las 20:00 UTC.
-> No se encontró la Banda 13 para 2011-11-20 a las 20:00 UTC.
-> No se encontró la Banda 13 para 2012-10-29 a las 11:00 UTC.
-> No se encontró la Banda 13 para 2013-02-09 a las 01:00 UTC.
-> No se encontró la Banda 13 para 2014-10-26 a las 01:00 UTC.
-> No se encontró la Banda 13 para 2014-12-09 a las 20:00 UTC.
-> No se encontró la Banda 13 para 2014-12-16 a las 05:00 UTC.
-> No se encontró la Banda 13 para 2015-09-25 a las 20:00 UTC.
-> No se encontró la Banda 13 para 2016-03-01 a las 01:00 UTC.
-> Descargando: ABI-L2-CMIPF/2017/095/20/OR_ABI-L2-CMIP

## Utilización para el proyecto

A continuación se creará una lista de eventos sobre el granizo, para luego descargar las imágenes satelitales correspondientes.
Esto permitirá generar un dataset con imágenes satelitales que tengan la etiqueta granizo (1=Si / 0=No).
Para su posterior uso y entrenamiento con una CNN

1) Importación de librerías 

In [11]:
import pandas as pd
import numpy as np

2) Cargar los datasets necesarios

In [12]:
# El dataset enriquecido, que tiene TODAS las fechas
ruta_dataset_enriquecido = "../data/processed/dataset_final_enriquecido.csv"
df_completo = pd.read_csv(ruta_dataset_enriquecido, parse_dates=['date'])

# El archivo de eventos, que tiene las HORAS que investigaste
ruta_eventos = "../data/raw/eventos_granizo_limpio_hora.csv"
df_eventos = pd.read_csv(ruta_eventos, parse_dates=['fecha'])

print("✅ Datasets cargados.")

✅ Datasets cargados.


2) Preparar las "Tareas Positivas" (Días con Granizo) 

In [13]:
# Nos quedamos con las columnas que necesitamos de tu investigación
tareas_positivas = df_eventos[['fecha', 'hora_aprox']].copy()
tareas_positivas = tareas_positivas.rename(columns={'fecha': 'date'})
tareas_positivas['granizo'] = 1
print(f"Se procesarán {len(tareas_positivas)} eventos de granizo (positivos).")

Se procesarán 84 eventos de granizo (positivos).


3) Preparar las "Tareas Negativas" (Días SIN Granizo)


In [14]:
# Filtramos los días que NO tuvieron granizo en nuestro dataset principal
df_sin_granizo = df_completo[df_completo['granizo'] == 0]

In [16]:
# Seleccionamos una muestra aleatoria (ej. 300 días) para tener un balance
# random_state es para que la muestra sea siempre la misma si volvemos a correr el código
tareas_negativas = df_sin_granizo.sample(n=300, random_state=42).copy()
# Les asignamos una hora estándar de la tarde (20:00 UTC), que es el pico de actividad
tareas_negativas['hora_aprox'] = "17:00 hs" 
tareas_negativas['granizo'] = 0
# Nos quedamos solo con las columnas que necesitamos
tareas_negativas = tareas_negativas[['date', 'hora_aprox', 'granizo']]
print(f"Se seleccionaron {len(tareas_negativas)} días sin granizo (negativos).")

Se seleccionaron 300 días sin granizo (negativos).


4) Unir Todo en una Única Lista de Tareas

In [17]:
df_tareas_imagenes = pd.concat([tareas_positivas, tareas_negativas], ignore_index=True)
# Mezclamos las filas para que no estén todas las positivas juntas
df_tareas_imagenes = df_tareas_imagenes.sample(frac=1, random_state=42).reset_index(drop=True)

print("\n\n✅ Lista de tareas de descarga de imágenes creada y mezclada.")
display(df_tareas_imagenes.head())
print(f"Total de imágenes a procesar: {len(df_tareas_imagenes)}")



✅ Lista de tareas de descarga de imágenes creada y mezclada.


,date,hora_aprox,granizo
0,2004-10-01,17:00 hs,0
1,2005-12-12,17:00 hs,0
2,2019-05-20,17:00 hs,0
3,2019-08-11,17:00 hs,0
4,2021-02-17,Tarde (aprox. 18:00 hs),1


Total de imágenes a procesar: 384


Importacion de librerias

In [ ]:
import pandas as pd
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import os
import re
import time

In [18]:
# --- 1. La Función "Traductora" de Horas (la misma que ya teníamos) ---
def parse_hora_a_utc(texto_hora):
    if pd.isna(texto_hora): return None
    texto_hora = str(texto_hora).lower()
    if 'utc' in texto_hora:
        numeros = re.findall(r'\d+', texto_hora)
        return int(numeros[0]) if numeros else None
    if 'tarde' in texto_hora: return 20
    if 'noche' in texto_hora: return 1
    if 'madrugada' in texto_hora: return 5
    if 'mediodía' in texto_hora or 'mediodia' in texto_hora: return 15
    numeros = re.findall(r'\d+', texto_hora)
    if numeros:
        hora_local = int(numeros[0])
        return (hora_local + 3) % 24
    return None

# --- 2. Configuración para la descarga ---
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))
bucket_name = 'noaa-goes16'
ruta_base_guardado = "../data/raw/imagenes_satelitales/"
os.makedirs(ruta_base_guardado, exist_ok=True)

print("\nIniciando la descarga de secuencias de imágenes satelitales...")
print("Este proceso puede tardar varias horas.")

# --- 3. Bucle principal de descarga de secuencias ---
for index, tarea in df_tareas_imagenes.iterrows():
    fecha = tarea['date']
    hora_central_utc = parse_hora_a_utc(tarea['hora_aprox'])
    
    if hora_central_utc is not None:
        # --- Lógica de Checkpoints por Carpeta ---
        nombre_carpeta_evento = f"{ruta_base_guardado}{fecha.strftime('%Y-%m-%d')}_{('granizo' if tarea['granizo'] == 1 else 'no_granizo')}/"
        
        # Si la carpeta existe y ya tiene 5 imágenes, la saltamos.
        if os.path.exists(nombre_carpeta_evento) and len(os.listdir(nombre_carpeta_evento)) >= 5:
            print(f"\nSecuencia para {fecha.strftime('%Y-%m-%d')} ya completa. Saltando.")
            continue
        
        os.makedirs(nombre_carpeta_evento, exist_ok=True)
        print(f"\nProcesando evento: {fecha.strftime('%Y-%m-%d')}")
        
        # --- Bucle para descargar la secuencia (-2h a +2h) ---
        for i in range(-2, 3):
            hora_utc_actual = (hora_central_utc + i) % 24
            
            try:
                año = fecha.year
                dia_del_año = fecha.dayofyear
                prefix = f"ABI-L2-CMIPF/{año}/{dia_del_año:03d}/{hora_utc_actual:02d}/"
                
                response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
                
                archivo_encontrado = None
                if 'Contents' in response:
                    for file in response['Contents']:
                        if 'C13' in file['Key']: # Buscamos la Banda 13
                            archivo_encontrado = file['Key']
                            break
                
                if archivo_encontrado:
                    nombre_archivo_local = f"{nombre_carpeta_evento}{fecha.strftime('%Y-%m-%d')}_{hora_utc_actual:02d}00_UTC_C13.nc"
                    if not os.path.exists(nombre_archivo_local):
                        print(f"  -> Descargando imagen para las {hora_utc_actual:02d}:00 UTC...")
                        s3.download_file(bucket_name, archivo_encontrado, nombre_archivo_local)
                    # No imprimimos "archivo ya existe" para no llenar la salida
                else:
                    print(f"  -> No se encontraron imágenes para las {hora_utc_actual:02d}:00 UTC.")

            except Exception as e:
                print(f"  -> ERROR procesando la hora {hora_utc_actual:02d}:00 UTC. Error: {e}")
            
            time.sleep(1) # Pausa entre cada hora

print("\n--- Proceso de descarga de secuencias finalizado ---")


Iniciando la descarga de secuencias de imágenes satelitales...
Este proceso puede tardar varias horas.

Procesando evento: 2004-10-01
  -> No se encontraron imágenes para las 18:00 UTC.
  -> No se encontraron imágenes para las 19:00 UTC.
  -> No se encontraron imágenes para las 20:00 UTC.
  -> No se encontraron imágenes para las 21:00 UTC.
  -> No se encontraron imágenes para las 22:00 UTC.

Procesando evento: 2005-12-12
  -> No se encontraron imágenes para las 18:00 UTC.
  -> No se encontraron imágenes para las 19:00 UTC.
  -> No se encontraron imágenes para las 20:00 UTC.
  -> No se encontraron imágenes para las 21:00 UTC.
  -> No se encontraron imágenes para las 22:00 UTC.

Procesando evento: 2019-05-20
  -> Descargando imagen para las 18:00 UTC...
  -> Descargando imagen para las 19:00 UTC...
  -> Descargando imagen para las 20:00 UTC...
  -> Descargando imagen para las 21:00 UTC...
  -> Descargando imagen para las 22:00 UTC...

Procesando evento: 2019-08-11
  -> Descargando image

KeyboardInterrupt: 

### Reconocer cantidad disponible de imágenes para días con granizo

Importación de librerías

In [1]:
import pandas as pd
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import re

Script para traduccion de horas, comprobacion de resultados mediante aws

In [2]:
# --- 1. Cargar tu lista de eventos y configurar la conexión ---
ruta_eventos = "../data/raw/eventos_granizo_limpio_hora.csv"
df_eventos = pd.read_csv(ruta_eventos, parse_dates=['fecha'])

s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))
bucket_name = 'noaa-goes16'

# --- 2. La Función "Traductora" de Horas ---
def parse_hora_a_utc(texto_hora):
    if pd.isna(texto_hora): return None
    texto_hora = str(texto_hora).lower()
    if 'utc' in texto_hora:
        numeros = re.findall(r'\d+', texto_hora)
        return int(numeros[0]) if numeros else None
    if 'tarde' in texto_hora: return 20
    if 'noche' in texto_hora: return 1
    if 'madrugada' in texto_hora: return 5
    if 'mediodía' in texto_hora or 'mediodia' in texto_hora: return 15
    numeros = re.findall(r'\d+', texto_hora)
    if numeros:
        hora_local = int(numeros[0])
        return (hora_local + 3) % 24
    return None

# --- 3. Misión de Reconocimiento ---
print("Iniciando misión de reconocimiento para eventos de granizo...")
eventos_descargables = []

# Filtramos solo los eventos a partir de 2017, que es cuando el satélite empezó a operar
for index, evento in df_eventos[df_eventos['fecha'].dt.year >= 2017].iterrows():
    hora_utc = parse_hora_a_utc(evento['hora_aprox'])
    
    if hora_utc is not None:
        fecha = evento['fecha']
        prefix = f"ABI-L2-CMIPF/{fecha.year}/{fecha.dayofyear:03d}/{hora_utc:02d}/"
        
        response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
        
        # Verificamos si se encontró algún archivo de la Banda 13
        if 'Contents' in response and any('C13' in file['Key'] for file in response['Contents']):
            print(f"  -> ✅ Imagen disponible para el {fecha.strftime('%Y-%m-%d')}")
            eventos_descargables.append(evento)
        else:
            print(f"  -> ❌ Imagen NO disponible para el {fecha.strftime('%Y-%m-%d')}")

# --- 4. Informe Final de la Misión ---
df_eventos_descargables = pd.DataFrame(eventos_descargables)
total_eventos_positivos = len(df_eventos_descargables)
print(f"\n\n--- Informe de Reconocimiento ---")
print(f"✅ Se encontraron {total_eventos_positivos} eventos de granizo con imágenes satelitales disponibles.")

Iniciando misión de reconocimiento para eventos de granizo...
  -> ✅ Imagen disponible para el 2017-04-05
  -> ✅ Imagen disponible para el 2020-11-12
  -> ✅ Imagen disponible para el 2022-02-23
  -> ✅ Imagen disponible para el 2024-02-28
  -> ✅ Imagen disponible para el 2017-03-26
  -> ✅ Imagen disponible para el 2018-11-25
  -> ✅ Imagen disponible para el 2018-12-19
  -> ✅ Imagen disponible para el 2021-12-16
  -> ✅ Imagen disponible para el 2023-03-08
  -> ✅ Imagen disponible para el 2023-11-28
  -> ✅ Imagen disponible para el 2024-01-25
  -> ✅ Imagen disponible para el 2017-03-25
  -> ✅ Imagen disponible para el 2018-11-04
  -> ✅ Imagen disponible para el 2018-11-29
  -> ✅ Imagen disponible para el 2018-12-29
  -> ✅ Imagen disponible para el 2019-01-01
  -> ✅ Imagen disponible para el 2019-03-31
  -> ✅ Imagen disponible para el 2020-02-08
  -> ✅ Imagen disponible para el 2021-02-17
  -> ✅ Imagen disponible para el 2021-04-20
  -> ✅ Imagen disponible para el 2024-01-11
  -> ❌ Imagen 

### Descarga de imagenes para eventos disponibles

Importar librerías 

In [3]:
import pandas as pd
import numpy as np
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import re
import time
import os

Este script realiza 3 pasos:
1) Cargar el Dataset Completo
2) Crea la Lista de Tareas Final y Balanceada
3) Script de Descarga de Secuencias

In [8]:
# ==============================================================================
# FASE 1: MISIÓN DE RECONOCIMIENTO
# ==============================================================================
print("--- Fase 1: Misión de Reconocimiento ---")

# --- Función "Traductora" de Horas (definida una sola vez) ---
def parse_hora_a_utc(texto_hora):
    if pd.isna(texto_hora): return None
    texto_hora = str(texto_hora).lower()
    if 'utc' in texto_hora:
        numeros = re.findall(r'\d+', texto_hora)
        return int(numeros[0]) if numeros else None
    if 'tarde' in texto_hora: return 20
    if 'noche' in texto_hora: return 1
    if 'madrugada' in texto_hora: return 5
    if 'mediodía' in texto_hora or 'mediodia' in texto_hora: return 15
    numeros = re.findall(r'\d+', texto_hora)
    if numeros:
        hora_local = int(numeros[0])
        return (hora_local + 3) % 24
    return None

# --- Cargar eventos y configurar conexión ---
try:
    ruta_eventos = "../data/raw/eventos_granizo_limpio_hora.csv"
    df_eventos_original = pd.read_csv(ruta_eventos, parse_dates=['fecha'])
    
    s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))
    bucket_name = 'noaa-goes16'

    eventos_descargables = []
    # Filtramos solo eventos a partir de 2017
    for index, evento in df_eventos_original[df_eventos_original['fecha'].dt.year >= 2017].iterrows():
        hora_utc = parse_hora_a_utc(evento['hora_aprox'])
        if hora_utc is not None:
            fecha = evento['fecha']
            prefix = f"ABI-L2-CMIPF/{fecha.year}/{fecha.dayofyear:03d}/{hora_utc:02d}/"
            response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
            if 'Contents' in response and any('C13' in file['Key'] for file in response['Contents']):
                eventos_descargables.append(evento)

    df_eventos_descargables = pd.DataFrame(eventos_descargables)
    total_eventos_positivos = len(df_eventos_descargables)
    print(f"✅ Reconocimiento finalizado. Se encontraron {total_eventos_positivos} eventos de granizo con imágenes disponibles.")

except FileNotFoundError:
    print(f"❌ Error: No se encontró el archivo '{ruta_eventos}'. No se puede continuar.")
    total_eventos_positivos = 0


# ==============================================================================
# FASE 2: CREAR LISTA DE TAREAS
# ==============================================================================
if total_eventos_positivos > 0:
    print("\n--- Fase 2: Creando lista de tareas final... ---")
    ruta_dataset_enriquecido = "../data/processed/dataset_final_enriquecido.csv"
    df_completo = pd.read_csv(ruta_dataset_enriquecido, parse_dates=['date'])

    tareas_positivas = df_eventos_descargables[['fecha', 'hora_aprox']].copy()
    tareas_positivas = tareas_positivas.rename(columns={'fecha': 'date'})
    tareas_positivas['granizo'] = 1

    df_sin_granizo = df_completo[(df_completo['granizo'] == 0) & (df_completo['date'].dt.year >= 2017)]
    tareas_negativas = df_sin_granizo.sample(n=total_eventos_positivos, random_state=42).copy()
    tareas_negativas['hora_aprox'] = "17:00 hs"
    tareas_negativas = tareas_negativas[['date', 'hora_aprox', 'granizo']]

    df_tareas_imagenes = pd.concat([tareas_positivas, tareas_negativas], ignore_index=True)
    df_tareas_imagenes = df_tareas_imagenes.sample(frac=1, random_state=42).reset_index(drop=True)

    print(f"✅ Lista de tareas final creada con {len(tareas_positivas)} eventos positivos y {len(tareas_negativas)} negativos.")
else:
    df_tareas_imagenes = pd.DataFrame()


# ==============================================================================
# FASE 3: DESCARGA DE SECUENCIAS
# ==============================================================================
if not df_tareas_imagenes.empty:
    ruta_base_guardado = "../data/raw/imagenes_satelitales/"
    os.makedirs(ruta_base_guardado, exist_ok=True)
    print("\n--- Fase 3: Iniciando la descarga de secuencias de imágenes satelitales ---")
    print("Este proceso puede tardar varios minutos.")

    for index, tarea in df_tareas_imagenes.iterrows():
        fecha = tarea['date']
        hora_central_utc = parse_hora_a_utc(tarea['hora_aprox'])
        
        if hora_central_utc is not None:
            nombre_carpeta_evento = f"{ruta_base_guardado}{fecha.strftime('%Y-%m-%d')}_{('granizo' if tarea['granizo'] == 1 else 'no_granizo')}/"
            if os.path.exists(nombre_carpeta_evento) and len(os.listdir(nombre_carpeta_evento)) >= 5:
                print(f"\nSecuencia para {fecha.strftime('%Y-%m-%d')} ya completa. Saltando.")
                continue
            
            os.makedirs(nombre_carpeta_evento, exist_ok=True)
            print(f"\nProcesando evento: {fecha.strftime('%Y-%m-%d')}")
            
            for i in range(-2, 3):
                hora_utc_actual = (hora_central_utc + i + 24) % 24 # Se asegura que el resultado sea siempre positivo
                try:
                    año = fecha.year
                    dia_del_año = fecha.dayofyear
                    prefix = f"ABI-L2-CMIPF/{año}/{dia_del_año:03d}/{hora_utc_actual:02d}/"
                    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
                    
                    archivo_encontrado = None
                    if 'Contents' in response:
                        for file in response['Contents']:
                            if 'C13' in file['Key']:
                                archivo_encontrado = file['Key']
                                break
                    
                    if archivo_encontrado:
                        nombre_archivo_local = f"{nombre_carpeta_evento}{fecha.strftime('%Y-%m-%d')}_{hora_utc_actual:02d}00_UTC_C13.nc"
                        if not os.path.exists(nombre_archivo_local):
                            print(f"  -> Descargando imagen para las {hora_utc_actual:02d}:00 UTC...")
                            s3.download_file(bucket_name, archivo_encontrado, nombre_archivo_local)
                    else:
                        print(f"  -> No se encontraron imágenes para las {hora_utc_actual:02d}:00 UTC.")
                except Exception as e:
                    print(f"  -> ERROR procesando la hora {hora_utc_actual:02d}:00 UTC. Error: {e}")
                time.sleep(1)

    print("\n--- Proceso de descarga de secuencias finalizado ---")
else:
    print("\nNo se ejecutó la Fase 3 porque no se generaron tareas de descarga.")

--- Fase 1: Misión de Reconocimiento ---
✅ Reconocimiento finalizado. Se encontraron 23 eventos de granizo con imágenes disponibles.

--- Fase 2: Creando lista de tareas final... ---
✅ Lista de tareas final creada con 23 eventos positivos y 23 negativos.

--- Fase 3: Iniciando la descarga de secuencias de imágenes satelitales ---
Este proceso puede tardar varios minutos.

Procesando evento: 2019-02-21
  -> Descargando imagen para las 18:00 UTC...
  -> Descargando imagen para las 19:00 UTC...
  -> Descargando imagen para las 20:00 UTC...
  -> Descargando imagen para las 21:00 UTC...
  -> Descargando imagen para las 22:00 UTC...

Procesando evento: 2020-09-16
  -> Descargando imagen para las 18:00 UTC...
  -> Descargando imagen para las 19:00 UTC...
  -> Descargando imagen para las 20:00 UTC...
  -> Descargando imagen para las 21:00 UTC...
  -> Descargando imagen para las 22:00 UTC...

Procesando evento: 2017-06-08
  -> Descargando imagen para las 18:00 UTC...
  -> Descargando imagen par